In [1]:
import requests
import bs4
import pandas as pd
import datetime as dt

In [10]:


player_id = "ferguca01"
url = 'https://www.baseball-reference.com/players/' \
    + player_id[0] + '/' + player_id + '.shtml'

print(str(url))

res = requests.get(url)
soup=bs4.BeautifulSoup(res.text, 'html.parser')
new_soup_text = soup.find('h2',text='Transactions').findNext('div', class_ = 'placeholder').next_sibling.next_sibling
new_soup = bs4.BeautifulSoup(new_soup_text, 'html.parser')

free_agent = False

all_p = new_soup('p')
arrival_dt = all_p[0].text.split(': ')[0]
arrival_method = all_p[0].text.split(': ')[1].split()[0]


for s in all_p:
    if 'Free Agent' in s.text or 'free agent' in s.text:
        free_agent = True
        free_agent_dt = s.text.split(': ')[-2]
        break
    if s.text.split(': ')[-1] == "Granted Free Agency.":
        free_agent = True
        free_agent_dt = s.text.split(': ')[-2]
        break

if not free_agent:
    free_agent_dt = 'NA'

print('arrival date:',arrival_dt)
print('arrival method:', arrival_method)

if arrival_method == 'Drafted':
    draft = all_p[0].find_all('a', href=True)
    drafted_by = draft[0].text
    draft_round = draft[1].text
    
    print('drafted by:', drafted_by)
    print('draft round:', draft_round)
    
print('free agent date:', free_agent_dt)



https://www.baseball-reference.com/players/f/ferguca01.shtml
arrival date: July 3, 2014
arrival method: Drafted
drafted by: Los Angeles Dodgers
draft round: 38th round
free agent date: NA


# if signed as free agent, then assign free agent if either signed before last CBA (before 2017) or after last CBA and above 25 years old

# if drafted, assign free agent after sixth season, if not granted free agency before

# if not assigned a free agency date, assign that date after the sixth season (future date)

In [7]:
df = pd.read_csv('meta_test.csv', usecols=['NAME', 'NAME_KEY', 'BIRTHDATE','YEAR'])
#df = df.sample(20)
df = df[['NAME', 'NAME_KEY', 'BIRTHDATE','YEAR']]
df['YEAR'] = df['YEAR'].astype(str)
df['concat_cols'] = df.apply(lambda x:'_'.join(x).split('_'), axis=1)
df

,NAME,NAME_KEY,BIRTHDATE,YEAR,concat_cols
0,Shohei Ohtani,ohtansh01,1994-07-05,2018,"[Shohei Ohtani, ohtansh01, 1994-07-05, 2018]"
1,Mike Lansing,lansimi01,1968-04-03,2000,"[Mike Lansing, lansimi01, 1968-04-03, 2000]"
2,Mike Lansing,lansimi01,1968-04-03,2000,"[Mike Lansing, lansimi01, 1968-04-03, 2000]"
3,Mike Lansing,lansimi01,1968-04-03,2001,"[Mike Lansing, lansimi01, 1968-04-03, 2001]"
4,Caleb Ferguson,ferguca01,1996-07-02,2018,"[Caleb Ferguson, ferguca01, 1996-07-02, 2018]"
5,Caleb Ferguson,ferguca01,1996-07-02,2019,"[Caleb Ferguson, ferguca01, 1996-07-02, 2019]"
6,Adolis Garcia,garciad02,1993-03-02,2018,"[Adolis Garcia, garciad02, 1993-03-02, 2018]"
7,Randy Dobnak,dobnara01,1995-01-17,2019,"[Randy Dobnak, dobnara01, 1995-01-17, 2019]"
8,Michel Baez,baezmi01,1996-01-21,2019,"[Michel Baez, baezmi01, 1996-01-21, 2019]"


In [9]:
for item in df.concat_cols:

    player_id = item[1]
    url = 'https://www.baseball-reference.com/players/' \
        + player_id[0] + '/' + player_id + '.shtml'

   

    res = requests.get(url)
    soup=bs4.BeautifulSoup(res.text, 'html.parser')
    new_soup_text = soup.find('h2',text='Transactions').findNext('div', class_ = 'placeholder').next_sibling.next_sibling
    new_soup = bs4.BeautifulSoup(new_soup_text, 'html.parser')

    free_agent = False

    all_p = new_soup('p')


    for s in all_p:

        if 'Free Agent' in s.text or 'free agent' in s.text:
            free_agent = True
            free_agent_dt = s.text.split(': ')[-2]
            free_agent_dt = dt.datetime.strptime(free_agent_dt, '%B %d, %Y').date()
            break
        if s.text.split(': ')[-1] == "Granted Free Agency.":
            free_agent = True
            free_agent_dt = s.text.split(': ')[-2]
            free_agent_dt = dt.datetime.strptime(free_agent_dt, '%B %d, %Y').date()
            break

    if not free_agent:
        free_agent_dt = 'restricted'

    for s in all_p:
        if 'Drafted' in s.text and 'Player signed' not in s.text:
            continue

        elif 'Drafted' in s.text and 'Player signed' in s.text:
            line = s.find_all('a', href=True)

            arrival_method = s.text.split(': ')[1].split()[0]
            arrival_dt = s.text.split(': ')[0] 
            arrival_dt = dt.datetime.strptime(arrival_dt, '%B %d, %Y').date()
            signed_dt = s.text.split('Player signed ')[1].replace('.','') 
            signed_dt = dt.datetime.strptime(signed_dt, '%B %d, %Y').date()
            draft_round = line[1].text
            break

        else:
            arrival_method = s.text.split(': ')[1].split()[0]
            arrival_dt = s.text.split(': ')[0]  
            arrival_dt = dt.datetime.strptime(arrival_dt, '%B %d, %Y').date()
            signed_dt = s.text.split(': ')[0]  
            signed_dt = dt.datetime.strptime(signed_dt, '%B %d, %Y').date()
            draft_round = 'not drafted'
            break

    if all_p[0].text.split(': ')[1].split()[0] == 'Drafted':
        was_drafted = 'yes'
    else:
        was_drafted = 'no'

    date1 = dt.datetime.strptime(str(signed_dt), '%Y-%m-%d')
    date2 = dt.datetime.strptime(item[2], '%Y-%m-%d')
    age_when_signed = int((date1 - date2).days / 365.2425)
    
    cond1 = dt.datetime.strptime(str(signed_dt), '%Y-%m-%d').date()>= dt.datetime.strptime('2016-12-14', '%Y-%m-%d').date()
    cond2 = arrival_method != 'Drafted' 
    cond3 = age_when_signed <25
    
    if  cond1 and cond2 and cond3:
        free_agent_dt = 'restricted'
    
    print(item[0])
    print(str(url))
    print('arrival date:',str(arrival_dt))
    print('arrival method:', arrival_method) 
    print('was drafted:', was_drafted)
    print('draft round:', draft_round)
    print('birthdate:', str(item[2]))
    print('signed date:',str(signed_dt))  
    print('age when signed:', age_when_signed)
    print('free agent date:', str(free_agent_dt))
    print('------------------------------------------\n')



Shohei Ohtani
https://www.baseball-reference.com/players/o/ohtansh01.shtml
arrival date: 2017-12-09
arrival method: Signed
was drafted: no
draft round: not drafted
birthdate: 1994-07-05
signed date: 2017-12-09
age when signed: 23
free agent date: restricted
------------------------------------------



IndexError: list index out of range

In [201]:
dt.date.today()

datetime.date(2020, 1, 2)

In [224]:
date1 = dt.datetime.strptime('1986-04-25', '%Y-%m-%d')
date2 = dt.datetime.strptime('2020-04-25', '%Y-%m-%d')
print(date1)
print(date2)

1986-04-25 00:00:00
2020-04-25 00:00:00


In [225]:
age = int((date2 - date1).days / 365.2425)
age

34

In [250]:
dt.datetime.strptime(signed_dt, '%Y-%m-%d')

TypeError: strptime() argument 1 must be str, not datetime.date

In [264]:
dt.date.strptime(str(signed_dt), '%Y-%m-%d') >= dt.date.strptime('2016-12-14', '%Y-%m-%d')

AttributeError: type object 'datetime.date' has no attribute 'strptime'

In [268]:
dt.datetime.strptime('2016-12-14', '%Y-%m-%d').date()

datetime.date(2016, 12, 14)

In [263]:
type(str(signed_dt))

str

In [269]:
dt.datetime.strptime(str(signed_dt), '%Y-%m-%d').date()>= dt.datetime.strptime('2016-12-14', '%Y-%m-%d').date()

False

In [280]:
opening_dates_dict = {'2000': '2000-03-29',
                      '2001': '2001-04-01',
                    '2002': '2002-03-31',
                     '2003': '2003-03-30',
                     '2004': '2004-03-30',
                     '2005': '2005-04-03',
                     '2006': '2006-04-02',
                     '2007': '2007-04-01',
                     '2008': '2008-03-25',
                     '2009': '2009-04-05',
                     '2010': '2010-04-04',
                     '2011': '2011-03-31',
                     '2012': '2012-03-28',
                     '2013': '2013-03-31',
                     '2014': '2014-03-22',
                     '2015': '2015-04-05',
                     '2016': '2016-04-03',
                     '2017': '2017-04-02',
                     '2018': '2018-03-29',
                     '2019': '2019-03-20'}

In [281]:
for date in range(2000,2020):
    print(opening_dates_dict[str(date)])

2000-03-29
2001-04-01
2002-03-31
2003-03-30
2004-03-30
2005-04-03
2006-04-02
2007-04-01
2008-03-25
2009-04-05
2010-04-04
2011-03-31
2012-03-28
2013-03-31
2014-03-22
2015-04-05
2016-04-03
2017-04-02
2018-03-29
2019-03-20


In [27]:
df = pd.read_csv('free_agents.csv')

In [12]:
df.head()

,NAME,NAME_KEY,YEAR,ARRIVAL_DT,ARRIVAL_METHOD,WAS_DRAFTED,DRAFT_ROUND,SIGNED_DT,BIRTHDATE,AGE_WHEN_SIGNED,FREE_AGENT_DT,IS_FREE_AGENT_NOW,SEASON_START_DT,PLAYER_URL
0,Bengie Molina,molinbe01,2000,1993-05-23,Signed,no,not drafted,1993-05-23,1974-07-20,18,1993-05-23,yes,2000-03-29,https://www.baseball-reference.com/players/m/m...
1,Mo Vaughn,vaughmo01,2000,1989-06-05,Drafted,yes,1st round,1989-06-20,1967-12-15,21,1998-10-23,yes,2000-03-29,https://www.baseball-reference.com/players/v/v...
2,Adam Kennedy,kennead01,2000,1997-06-03,Drafted,yes,1st round,1997-06-18,1976-01-10,21,2006-10-15,no,2000-03-29,https://www.baseball-reference.com/players/k/k...
3,Benji Gil,gilbe01,2000,1991-06-03,Drafted,yes,1st round,1991-07-19,1972-10-06,18,1999-10-15,yes,2000-03-29,https://www.baseball-reference.com/players/g/g...
4,Troy Glaus,glaustr01,2000,1997-06-03,Drafted,yes,1st round,1997-09-29,1976-08-03,21,2004-10-28,no,2000-03-29,https://www.baseball-reference.com/players/g/g...


In [13]:
df['ARRIVAL_METHOD'].unique()

array(['Signed', 'Drafted', 'Purchased', 'Traded', 'Granted', 'Returned',
       'Released', 'the', 'Selected'], dtype=object)

In [17]:
pd.set_option('display.max_colwidth', -1)

In [11]:
df2 = df[['NAME','NAME_KEY', 'YEAR']][(df['ARRIVAL_METHOD']!= 'Drafted') & (df['ARRIVAL_METHOD']!= 'Signed')]

In [12]:
df2.shape

(426, 3)

In [14]:
meta = pd.read_csv('meta.csv')

In [16]:
df2 = pd.merge(meta,df2, on = ['NAME', 'NAME_KEY', 'YEAR'], how = 'inner')

In [8]:
meta.columns

Index(['NAME', 'NAME_KEY', 'YEAR', 'TEAM', 'BATS', 'THROWS', 'HEIGHT',
       'HEIGHT_CM', 'WEIGHT_LB', 'WEIGHT_KG', 'BIRTHDATE', 'BIRTHPLACE'],
      dtype='object')

In [9]:
df2.columns

Index(['NAME', 'PLAYER_URL', 'YEAR'], dtype='object')

In [18]:
df2.shape

(520, 12)

In [20]:
df2 = df2.drop_duplicates()

In [21]:
df2.shape

(426, 12)

In [22]:
df2.to_csv('meta_test.csv', mode='w', header = True, index = False)

In [23]:
df.shape

(28140, 14)

In [28]:
df = df[(df['ARRIVAL_METHOD'] == 'Drafted') | (df['ARRIVAL_METHOD'] == 'Signed')]

In [29]:
28140 - df.shape[0]

426

In [30]:
df.to_csv('free_agents.csv', mode='w', header = True, index = False)